In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
import copy
from sklearn.model_selection import train_test_split
import torchvision
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
cdata = torchvision.datasets.EMNIST(root='emnist',split='letters',download=True)


In [ ]:
images = cdata.data.view([124800,1,28,28]).float()
# normalize the images
images /= torch.max(images)
print('\nTensor data:')
print(images.shape)


In [ ]:
letterCategories = cdata.classes[1:]
labels = copy.deepcopy(cdata.targets)-1

In [ ]:
train_data,test_data, train_labels,test_labels = train_test_split(images, labels, test_size=.1)
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)
batchsize    = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [ ]:
def makeTheNet(numchans=(6,6)):

  class emnistnet(nn.Module):
    def __init__(self,numchans):
      super().__init__()

      self.conv1  = nn.Conv2d(1,numchans[0],3,padding=1)
      self.bnorm1 = nn.BatchNorm2d(numchans[0])
      self.conv2  = nn.Conv2d(numchans[0],numchans[1],3,padding=1)
      self.bnorm2 = nn.BatchNorm2d(numchans[1])
      self.fc1 = nn.Linear(7*7*numchans[1],50)
      self.fc2 = nn.Linear(50,26)

    def forward(self,x):

      x = F.max_pool2d(self.conv1(x),2)
      x = F.leaky_relu(self.bnorm1(x))
      x = F.max_pool2d(self.conv2(x),2)
      x = F.leaky_relu(self.bnorm2(x))
      nUnits = x.shape.numel()/x.shape[0]
      x = x.view(-1,int(nUnits))
      x = F.leaky_relu(self.fc1(x))
      x = self.fc2(x)
      return x

  net = emnistnet(numchans)
  lossfun = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(net.parameters(),lr=.001)

  return net,lossfun,optimizer

In [ ]:
# a function that trains the model

def function2trainTheModel(numchans):

  numepochs = 5

  net,lossfun,optimizer = makeTheNet(numchans)

  net.to(device)

  trainLoss = torch.zeros(numepochs)
  testLoss  = torch.zeros(numepochs)
  trainErr  = torch.zeros(numepochs)
  testErr   = torch.zeros(numepochs)


  for epochi in range(numepochs):

    net.train()
    batchLoss = []
    batchErr  = []
    for X,y in train_loader:

      X = X.to(device)
      y = y.to(device)

      yHat = net(X)
      loss = lossfun(yHat,y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      batchLoss.append(loss.item())
      batchErr.append( torch.mean((torch.argmax(yHat,axis=1) != y).float()).item() )

    trainLoss[epochi] = np.mean(batchLoss)
    trainErr[epochi]  = 100*np.mean(batchErr)
    net.eval()
    X,y = next(iter(test_loader)) 

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
      yHat = net(X)
      loss = lossfun(yHat,y)

    testLoss[epochi] = loss.item()
    testErr[epochi]  = 100*torch.mean((torch.argmax(yHat,axis=1) != y).float()).item()


  return trainLoss,testLoss,trainErr,testErr,net

In [ ]:
trainLoss,testLoss,trainErr,testErr,net = function2trainTheModel((3,7))
fig,ax = plt.subplots(1,2,figsize=(15,3))

ax[0].plot(trainLoss,'s-',label='Train')
ax[0].plot(testLoss,'o-',label='Test')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss ')
ax[0].set_title('Model loss')

ax[1].plot(trainErr,'s-',label='Train')
ax[1].plot(testErr,'o-',label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Error rates')
ax[1].set_title(f'Last model test error rate: {testErr[-1]:.2f}%')
ax[1].legend()

plt.show()

In [ ]:
convChannelsList=[2,5,8]
results=torch.zeros(len(convChannelsList),len(convChannelsList),2)
totalChanns=torch.zeros(len(convChannelsList),len(convChannelsList))
'''
 a tensor with shape 3,3,2 means 3 arrays each has 3 arrays and from the last each array has 2 elements
# 3 3 2
each has 2 elements
[[],[],[]]
 [[],[],[]]
 [[],[],[]]


  '''
for i,firstElConv in enumerate(convChannelsList):
  for j,secondElConv in enumerate(convChannelsList):
    trainLoss,testLoss,trainErr,testErr,net = function2trainTheModel((firstElConv,secondElConv))

    results[i,j,:]=testErr[-1],trainErr[-1]
    totalChanns[i,j]=firstElConv+secondElConv
